## Correlated Underlyings and Their effect on Option Payoffs

In this exercise we will analyze three types of options with different payoff structures and determine how differences in correlation between the underlying affect the payoff.

Let us consider two stocks, determined by their respective processes $\{(S_1(t), S_2(t)) : t \geq 0\}$ be geometric brownian motions with drift vector $\boldsymbol{\mu}$ and covariance matrix $\boldsymbol{\Sigma}$.

In this example, let us assume that $\boldsymbol{\mu} = [r, r]^T$ and, since all covariance matricies are real and positive semi-definite, $\boldsymbol{\Sigma}$ can be decomposed as $\boldsymbol{\Sigma} = AA^T$

If we take $A$ to be a lower triangular matrix, then this is called a Cholesky decomposition. However, I am going to do something simpler. Since this problem only involves two variables, I will set $A = \begin{pmatrix}
1 & c\\
c & 1
\end{pmatrix} $ instead of $A = \begin{pmatrix}
1 & 0\\
c & 1
\end{pmatrix} $

The second formulation is what would be used for a cholesky decomposition.

You can verify for yourself that if we take the first definition of $A$, then the variances of each process will be equal, whereas in a cholesky decomposition, that is not necessarily true.

I will choose multiple values of $c$ to control the amount of covariance that each stock has with each other.

Regarding the options, we will test three options. They and their payoffs are:

1. Spread option: $ Y_1 = \mathbb{E}^\mathbb{Q} \left[ e^{-r(T-t)}\max(S_1(T) - S_2(T) - K, 0)  | \mathcal{F}_t \right] $
2. Basket option: $ Y_1 = \mathbb{E}^\mathbb{Q} \left[ e^{-r(T-t)}\max(S_1(T) + S_2(T) - K, 0)  | \mathcal{F}_t \right] $
3. Outperformance option: $ Y_1 = \mathbb{E}^\mathbb{Q} \left[ e^{-r(T-t)}\max[\max(S_1(T) , S_2(T)) - K)]  | \mathcal{F}_t \right] $


For this example, we will choose parameters as follows

$S_1(0) = S_2(0) = 1, K = 2, r = 0.05, T = 1$
$c = -0.5, 0, 0.5$
We will use 20,000 monte carlo iterations and comment on our findings. Speaking of which, before we run any monte carlo simulations, simply by looking at the payoffs, we expect that when $S_1$ and $S_2$ diverge significantly, we expect a high payoff on the spread and a low payoff on the basket. Thus, when we believe stocks are negatively correlated, we prefer the spread option. Similarly, we expect that when $S_1$ and $S_2$ are highly correlated, we expect a high payoff on the basket and a low payoff on the spread. Thus, when we believe stocks are positively correlated, we prefer the basket option.

In [41]:
import numpy as np
import scipy.stats as sp
import matplotlib.pyplot as plt

# GBM parameters
r = 0.05
mu = np.array([r, r])
S0_1 = 1
S0_2 = 1

# Other parameters
K = 2; # strike

c_values = [-0.5, -0.25, 0, 0.25, 0.5]
Sigma_values = [0, 0, 0, 0, 0]

names = ['spread', 'basket', 'outperformance']
#cart_prod = [(a,b) for a in c_values for b in names]
#results = {key: None for key in c_values}
results = {}

#Generate 20000 sample paths
#Note that we are generating only one time point worth of brownians. That is, in our simulation, we have a value at T = 0 and a value at T = 1, from the random variable draw
#If we want multiple time points, we will need another dimension added to this draw

np.random.seed(seed=13818376)

reps = 20000 # of replications (sample paths)
T = 1

#standard normal RVs for W(t)
Z = sp.norm.rvs(loc = 0, scale = np.sqrt(T), size = (reps, 2))

#WE apply the common random numbers scheme to make sure we analyze only the affects of changing the parameter c
for i in range(0, len(c_values)):
    c = c_values[i]
    A = np.array([[1,c],[c,1]])
    Sigma_values[i] = np.matmul(A, np.transpose(A))
    Sigma = Sigma_values[i]

    ST_1 = S0_1*np.exp((mu[0] - Sigma[0,0]/2)*T + A[0,0]*Z[:,0] + A[0,1]*Z[:,1])
    ST_2 = S0_2*np.exp((mu[1] - Sigma[1,1]/2)*T + A[1,0]*Z[:,0] + A[1,1]*Z[:,1])

    #Confidence level critical value for 95% interval
    z_alpha2 = 1.96

    payoffs_spread = np.exp(-r*T)*np.maximum(ST_1 - ST_2 - K, 0)
    p_s_mean = np.mean(payoffs_spread)
    p_s_std = np.std(payoffs_spread)
    p_s_lb = p_s_mean - z_alpha2*p_s_std/np.sqrt(reps)
    p_s_ub = p_s_mean + z_alpha2*p_s_std/np.sqrt(reps)
    results[ (c, 'spread')] = [p_s_mean, p_s_std, p_s_lb, p_s_ub]

    payoffs_basket = np.exp(-r*T)*np.maximum(ST_1 + ST_2 - K, 0);
    p_b_mean = np.mean(payoffs_basket);
    p_b_std = np.std(payoffs_basket);
    p_b_lb = p_b_mean - z_alpha2*p_b_std/np.sqrt(reps)
    p_b_ub = p_b_mean + z_alpha2*p_b_std/np.sqrt(reps)
    results[ (c, 'basket')] = [p_b_mean, p_b_std, p_b_lb, p_b_ub]

    payoffs_outperform = np.exp(-r*T)*np.maximum(np.maximum(ST_1, ST_2) - K, 0)
    p_o_mean = np.mean(payoffs_outperform)
    p_o_std = np.std(payoffs_outperform)
    p_o_lb = p_o_mean - z_alpha2*p_o_std/np.sqrt(reps)
    p_o_ub = p_o_mean + z_alpha2*p_o_std/np.sqrt(reps)
    results[ (c, 'outperformance')] = [p_o_mean, p_o_std, p_o_lb, p_o_ub]

Now let's visualize our results.

In [42]:
import pandas as pd
pd.set_option('display.precision', 4)

asdf = pd.DataFrame(data = results)
asdf.rename(index={0: "Mean", 1: "StDev", 2: "CI Lower Bound", 3: "CI Upper Bound"})

#Output = pd.DataFrame(data = asdf.transpose(), columns = ['Mean', 'StDev', 'CI Lower Bound', 'CI Upper Bound'])

-0.50                          -0.25                         \
                spread  basket outperformance  spread  basket outperformance   
Mean            0.2376  0.5649         0.5086  0.1828  0.5699         0.4350   
StDev           1.3737  1.8247         1.8071  1.0813  1.5458         1.4684   
CI Lower Bound  0.2186  0.5396         0.4835  0.1678  0.5485         0.4147   
CI Upper Bound  0.2567  0.5902         0.5336  0.1978  0.5914         0.4554   

                  0.00                           0.25                         \
                spread  basket outperformance  spread  basket outperformance   
Mean            0.1354  0.6344         0.3996  0.0928  0.7320         0.3883   
StDev           0.8843  1.5939         1.3392  0.7360  1.9615         1.3911   
CI Lower Bound  0.1231  0.6123         0.3810  0.0826  0.7048         0.3691   
CI Upper Bound  0.1477  0.6565         0.4181  0.1030  0.7592         0.4076   

                  0.50                         
                spread  basket outperformance  
Mean            0.0539  0.8457         0.3869  
StDev           0.5722  2.5901         1.5707  
CI Lower Bound  0.0460  0.8098         0.3651  
CI Upper Bound  0.0619  0.8815         0.4087

We notice a few things.

First, at the very left, when the correlation is the most negative, we see that the spread option is worth the most, at 0.2376. But as we increase the correlation (as we scan from the left to right on the top most row), the value of the spread option decreases, monotonically.

This is consistent with our intuition.

Similarly, at the most negative correlation, the basket option is worth the least, but it increases in value monotonically as we increase the correlation.

The outperformance option si tricky to understand. It has the highest expected payoff when the assets are negatively correlated. We can think of it this way, if the assets are positively correlated, then when we take $\max(S_1, S_2)$, if the values were going to be high to begin with, it doesn't matter which one is higher, so long as one clears the strike. If the values were low to begin with, then it wouldn't matter because the payoff would have been zero. However, when the correlation is strongly negative, then even if one path does not clear the strike price, the other path could. This phenomenon, over enough paths, manifests in the higher valuation.

Finally, we see that across the board, the Basket option is more valuable than the outpeformance option, which is more valuable than the spread option.